In [11]:
import numpy as np 
import glob 

def process_vtk_file(S_file, new_S_file):
    """
    Processes a VTK file to extract POINTS coordinates, validate the array, and write a new file with LINE and scalar data.

    Args:
        S_file (str): The path to the input VTK file.
        new_S_file (str): The path to the output VTK file.
    """
    import numpy as np

    # Read the input file
    with open(S_file, "r") as f:
        lines = f.readlines()

    # Locate the POINTS section
    points_start = None
    points_count = 0
    for i, line in enumerate(lines):
        if line.startswith("POINTS"):
            points_start = i
            points_count = int(line.split()[1])  # Extract the number of points
            break

    if points_start is None:
        raise ValueError("POINTS section not found in the input file.")

    # Extract POINTS data
    points_data = []
    for line in lines[points_start + 1:]:
        if line.startswith("LINES") or line.strip() == "":  # Stop at LINES or empty line
            break
        points_data.extend(map(float, line.split()))

    # Validate the number of coordinates
    if len(points_data) != points_count * 3:
        raise ValueError(f"Mismatch in POINTS data: expected {points_count * 3} values, got {len(points_data)}.")

    # Reshape into (points_count, 3)
    points_array = np.array(points_data).reshape(-1, 3)

    # Validate the shape is (1151, 3)
    if points_array.shape != (1151, 3):
        raise ValueError(f"POINTS section is not a 1151x3 array. Actual shape: {points_array.shape}")

    # Construct new content
    new_content = [
        "# vtk DataFile Version 2.0\n",
        "Vessel No.56\n",
        "ASCII\n",
        "DATASET POLYDATA\n",
        f"POINTS {points_count} float\n"
    ]

    # Write POINTS with each line containing 3 floats
    for point in points_array:
        new_content.append(f"{point[0]} {point[1]} {point[2]}\n")

    # Add LINES section
    total_line_entries = points_count + 1  # Total entries in LINES block
    new_content.append(f"LINES 1 {total_line_entries}\n")
    new_content.append(f"{points_count} " + " ".join(map(str, range(points_count))) + "\n")

    # Add scalar data
    new_content.append("POINT_DATA 1151\n")
    new_content.append("SCALARS MaximumInscribedSphereRadius float\n")
    new_content.append("LOOKUP_TABLE default\n")
    new_content.extend(["2.0\n"] * 1151)

    # Write to the new file
    with open(new_S_file, "w") as f:
        f.writelines(new_content)


filenames = ['BH0006_ICA_L','BH0039_ICA_L','BG0011_ICA_L','BH0031_ICA_L']

for filename in filenames:
    process_vtk_file(f"D:/!BraVa_src/ica_results/brava_ica_mirrored/{filename}.vtk", 
                 f"D:/!BraVa_src/newShape/{filename}.vtk")
# process_vtk_file("D:/!BraVa_src/ica_results/brava_ica_mirrored/BH0006_ICA_L.vtk", 
#                  "D:/!BraVa_src/newShape/BH0006_ICA_L.vtk")